# Exercises XP

1. Install Required Libraries

Ensure you have the necessary libraries installed, including gensim, spacy, torch, and scikit-learn.

In [ ]:
from torch.nn import Module, LSTM, Linear, Dropout, MSELoss
from torch.optim import Adam
from torch.utils.data import Dataset
from torch import Tensor, save, load
import gensim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2. Load and Preprocess the Dataset

    Download the stock market dataset.
    
    Drop unnecessary columns and create a target column for the next day’s closing price.
    
    Normalize the dataset using MinMaxScaler.

The Dataset is not the correct one. I'll use the one we used in class.

In [ ]:
dataset = pd.read_csv('all_stocks_2006-01-01_to_2018-01-01.csv', index_col='Date', parse_dates=['Date'])
dataset

In [ ]:
dataset = dataset.drop(columns=['Name', 'Volume'])

dataset['Target'] = dataset['Close'].shift(-1) # To create the Target column
dataset = dataset.dropna()

In [ ]:
for i in dataset.columns:
    print(dataset[i].dtype) # I want to check if I can scale the columns

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1))
dataset = sc.fit_transform(dataset) 

3. Prepare the Dataset for Training

    Split the dataset into training, validation, and testing sets.
    
    Create a custom PyTorch Dataset class to handle the data.
    
    Use DataLoader to create iterable datasets for training and evaluation.

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.2, shuffle=False)
train_data, val_data  = train_test_split(train_data, test_size=0.2, shuffle=False)

In [ ]:
# Define a function to create features and targets
def create_features_and_targets(data, lookback=60):
    X = []
    y = []
    
    for i in range(lookback, len(data)):
        X.append(data.iloc[i-lookback:i][['Open', 'High', 'Low', 'Close']].values)
        y.append(data.iloc[i]['Target'])
    
    return np.array(X), np.array(y)

# Prepare features and targets for training, validation, and testing sets
lookback = 60  # Number of previous days to look at for prediction

X_train, y_train = create_features_and_targets(train_data, lookback)
X_val, y_val = create_features_and_targets(val_data, lookback)
X_test, y_test = create_features_and_targets(test_data, lookback)

# Convert the features and targets to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)